
# Task 4 微调技术（选做）

## 前置知识

### 监督微调 (SFT)

- 明确SFT的目标，并熟悉其典型的数据格式。

使用高质量示例数据集，对预训练好的LLM进行微调，以在特定任务上表现更好。
```python
[
  {"role": "user", "content": ""},
  {"role": "assistant", "content": ""}
]
```

----


### LoRA 技术原理

#### 掌握LoRA通过低秩适配来模拟权重更新的核心思想。

[LoRA](./LoRA%20Low-Rank%20Adaptation%20of%20Large%20Language%20Models.md)

#### 理解其关键超参数的作用及其对模型训练的影响。

- $\alpha,r$：控制更新幅度，类似学习率。
- 使用位置：对哪些矩阵进行LoRA微调。

----

### 训练优化技术:

了解在计算资源有限时进行模型训练的常用技术，如梯度累积、混合精度训练或模型量化的工作原理。

- **梯度累积**：将数据分成多个小批次，依次计算每个小批次的梯度并进行累加，达到一定批次后再用累积的总梯度来更新一次模型参数 。使用较小的GPU内存来近似大批次训练。

- **混合精度训练**：让模型权重和梯度使用FP32（单精度）以保持数值稳定性，而激活值和中间计算则使用FP16（半精度）来提高计算效率。

- **模型量化**：将连续的浮点数值范围映射为离散的整数集。

- 层间并行、层内并行、交替稀疏注意力。[GPT3](./Language%20Models%20are%20Few-Shot%20Learners.md)

----

## 实践任务

利用 LoRA 技术对 `Qwen/Qwen2.5-0.5B-Instruct` 模型进行监督微调，使其能够准确回答关于《华中科技大学学生手册》的内容。

**数据集:** `alleyf/HUST-Student-Handbook` ([数据集](https://www.modelscope.cn/datasets/alleyf/HUST-Student-Handbook/files))，也可以用Task 3自己构建的数据集

- 在实际微调中，常会遇到因资源不足导致无法训练的问题。除了选择更小的基础模型外，请至少列举并说明**两种**可以在代码层面实现的、用于缓解显存压力的训练技巧，并简述它们的基本工作原理。

答案见上。

- 思考应该如何全面、客观地衡量其微调后的效果？尝试设计一个评估方案。
  - 双盲测试，由人类评估微调前后对对同一问题的回答质量，包括语言流畅性，与标准答案的一致性等。
  - 使用类似数据集，如《武汉大学学生手册》作为验证集，对微调后的模型进行评估。
  - 评估其计算资源的消耗。
  - 计算标准的NLP指标。如tokenize后的F1 score, Exact Match等
```plaintext
Precision = 共同词数 / 预测词总数  
Recall    = 共同词数 / 标准答案词总数  
F1 = 2 * (Precision * Recall) / (Precision + Recall)
```

----

## 数据集

```python
{
  "conversations": [
    {
      "role": "user",
      "content": "用户提问内容"
    },
    {
      "role": "assistant",
      "content": "助手回答内容"
    }
  ]
}
```

### 示例代码

In [1]:
from modelscope.msdatasets import MsDataset

dataset = MsDataset.load(
    './data/lora_hust_student_handbookt.jsonl',
    split='train',
    trust_remote_code=True
)

print(f"数据集包含 {len(dataset)} 组问答对")


def prepare_training_data(data):
    training_data = []
    for item in data:
        conversations = item["conversations"]
        training_data.append({
            "input": conversations[0]["content"],
            "output": conversations[1]["content"]
        })
    return training_data

training_data = prepare_training_data(dataset)
print(training_data[0])

d:\anaconda\envs\modscope\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-09-27 09:57:22,674 - modelscope - WARNING - Use trust_remote_code=True. Will invoke codes from ./data/lora_hust_student_handbookt.jsonl. Please make sure that you can trust the external codes.
2025-09-27 09:57:22,674 - modelscope - WARNING - Use trust_remote_code=True. Will invoke codes from json. Please make sure that you can trust the external codes.


数据集包含 719 组问答对
{'input': '华中科技大学博士生培养目标中，对学术能力和专业能力分别提出哪些具体要求？', 'output': '华中科技大学博士生培养目标中对学术能力和专业能力的具体要求如下：\n\n**学术能力要求**：\n- 掌握本学科或专业领域坚实宽广的基础理论和系统深入的专门知识；\n- 具有独立从事科学研究的能力；\n- 具备良好的写作和国际学术交流能力；\n- 能够在科学研究领域作出创新性研究。\n\n**专业能力要求**：\n- 具有独立承担专业工作的能力；\n- 对于专业学位博士生，重点培养其通过专业实践掌握独立承担专业工作的能力。\n\n这些要求旨在培养博士生在学术研究和专业实践中的高水平能力，确保他们成为德智体美劳全面发展的社会主义建设者和接班人。'}


## 模型

### [示例代码](https://www.modelscope.cn/models/qwen/Qwen2.5-0.5B-Instruct/summary)

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen2.5-0.5B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

prompt = "Give me a short introduction to large language model."
messages = [
    {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

### LoRA 微调

In [11]:
from modelscope import snapshot_download

model_dir = snapshot_download('qwen/Qwen2.5-0.5B-Instruct', cache_dir='./qwen')
print(model_dir)

2025-09-20 16:15:10,830 - modelscope - INFO - Creating symbolic link [./qwen\qwen\Qwen2.5-0.5B-Instruct].
2025-09-20 16:15:10,832 - modelscope - WARNING - Failed to create symbolic link ./qwen\qwen\Qwen2.5-0.5B-Instruct for d:\2nd\qwen\qwen\Qwen2___5-0___5B-Instruct.


./qwen\qwen\Qwen2___5-0___5B-Instruct


In [2]:
from modelscope.msdatasets import MsDataset
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model
import torch


dataset = MsDataset.load(
    './data/lora_hust_student_handbookt.jsonl',
    split='train',
    trust_remote_code=True
)
print(f"数据集包含 {len(dataset)} 组问答对")

def prepare_training_data(data):
    training_data = []
    for item in data:
        conversations = item["conversations"]
        training_data.append({
            "input": conversations[0]["content"],
            "output": conversations[1]["content"]
        })
    return training_data
data = prepare_training_data(dataset)


class SFTDataset(torch.utils.data.Dataset):
    def __init__(self, data, tokenizer, max_length=512):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        messages = [
            {"role": "user", "content": item["input"]},
            {"role": "assistant", "content": item["output"]}
        ]
        text = self.tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=False
        )
        tokenized = self.tokenizer(
            text,
            truncation=True,
            max_length=self.max_length,
            padding="max_length",
            return_tensors="pt"
        )
        input_ids = tokenized["input_ids"].squeeze()  # 去除所有维度为1的轴
        attention_mask = tokenized["attention_mask"].squeeze()
        return {"input_ids": input_ids, "attention_mask": attention_mask, "labels": input_ids}
        # "labels" 等于 input_ids，即自回归地预测


model_dir = './qwen./qwen/Qwen2___5-0___5B-Instruct'
tokenizer = AutoTokenizer.from_pretrained(model_dir, trust_remote_code=True)
base_model = AutoModelForCausalLM.from_pretrained(model_dir, trust_remote_code=True)

# LoRA配置
lora_config = LoraConfig(
    r=8,
    lora_alpha=8,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",           # 不对bias参数进行微调
    task_type="CAUSAL_LM"  # 任务类型：因果语言建模（自回归生成）
)
model = get_peft_model(base_model, lora_config)


train_data = SFTDataset(data, tokenizer)

training_args = TrainingArguments(
    per_device_train_batch_size=4,  # 每块设备上的batch size
    gradient_accumulation_steps=8,  # 每8步更新梯度，相当于batch size=32
    fp16=True,                      # 混合精度
    num_train_epochs=3,
    output_dir="./lora_output",
    logging_steps=20,               # 每20步记录一次日志
    save_steps=100,                 # 每100步保存一次模型
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
)

trainer.train()
trainer.save_model("./lora_output")

2025-09-27 10:19:17,197 - modelscope - WARNING - Use trust_remote_code=True. Will invoke codes from ./data/lora_hust_student_handbookt.jsonl. Please make sure that you can trust the external codes.
2025-09-27 10:19:17,198 - modelscope - WARNING - Use trust_remote_code=True. Will invoke codes from json. Please make sure that you can trust the external codes.


数据集包含 719 组问答对


Step,Training Loss
20,11.518900
40,10.052200
60,9.031400


### 问答

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

original_model_dir = './qwen./qwen/Qwen2___5-0___5B-Instruct'
finetuned_model_dir = './lora_output'

model = AutoModelForCausalLM.from_pretrained(  # 因果语言建模的预训练模型
    finetuned_model_dir,
    torch_dtype="auto",  # 自动选择数据类型
    device_map="auto",   # 自动将模型分配到可用的设备
    trust_remote_code=True
)
tokenizer = AutoTokenizer.from_pretrained(original_model_dir, trust_remote_code=True)


prompt = "华中科技大学学生请假流程是什么？"
messages = [
    {"role": "system", "content": "你是Qwen，华中科技大学学生手册智能助手。"},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(   # 将多轮对话自动格式化为模型需要的输入文本。
    messages,
    tokenize=False, # 只生成原始文本，不做分词。
    add_generation_prompt=True  # 在文本末尾添加生成提示，方便模型生成回复。
)  # 类似 <|user|>请假流程是什么？<|assistant|>请假流程如下…… 的普通字符串
model_inputs = tokenizer([text], return_tensors="pt").to(model.device) # 分词并转换为 PyTorch 张量
# model_inputs 字典包含两个 PyTorch 张量
# input_ids：分词后的 token 序列
# attention_mask：注意力掩码，和inputs_ids 等长


generated_ids = model.generate(  # 生成 token 序列
    **model_inputs,  # 解包出键值对
    max_new_tokens=512  # 最大回复长度
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
] # 从模型的完整输出中截取掉原始输入 prompt

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0] # 一个对话
print(response)

华中科技大学的学生请假流程如下：

1. **填写请假单**：首先，你需要填写《华中科技大学学生请假条》，并确保提交给学校相关部门或辅导员。

2. **申请请假**：你可以在规定的时间内通过学校的请假系统（如学工网、网上办公平台等）提交你的请假申请。通常需要提供以下信息：
   - 姓名和身份证号码。
   - 请假事由说明。
   - 具体的请假日期。
   - 是否有特殊情况需要延长假期。

3. **审批过程**：学校会根据实际情况进行审批，包括审核请假原因是否合理，是否有足够的理由来延长假期等。

4. **确认并执行**：如果批准了请假请求，你可以按照学校的指示去办理相关手续，并在规定的期限内完成请假。

5. **后续处理**：请务必遵守学校的规定，按时完成请假程序，以避免影响其他学生的正常学习生活。

请注意，具体的请假流程可能会有所变化，请提前查看最新的校规校纪或者直接联系学校相关部门获取最准确的信息。
